# Import Libraries

In [1]:
import optuna
import os
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [2]:
def read_file(*args,file_type="csv")->pd.DataFrame:
    """:parameter input directories in sequence
        :return DataFrame"""
    
    path=os.path.join(*args)
    if file_type == "tsv":
        return pd.read_csv(path, sep="\t")
    return pd.read_csv(path)

# Experiments

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [4]:
mlflow.set_experiment("SVC hyper-parameters tuning")

2024/10/19 13:29:13 INFO mlflow.tracking.fluent: Experiment with name 'SVC hyper-parameters tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/spynom/mlruns/15', creation_time=1729324753858, experiment_id='15', last_update_time=1729324753858, lifecycle_stage='active', name='SVC hyper-parameters tuning', tags={}>

In [5]:
df=read_file("..","data","processed","cleaned.csv").dropna(how="any").drop_duplicates()

X=df.comment
y=df.category+1

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
vectorizer.fit(X)  # Fit on all vocab data
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)  # Transform test data

In [6]:

class_weights = compute_class_weight('balanced', classes=np.array([0, 1, 2]), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

In [7]:
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_BalanceWeight_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        train_y_pred=model.predict(X_train)
        test_y_pred = model.predict(X_test)

        # Log accuracy
        train_accuracy = accuracy_score(y_train,train_y_pred)
        test_accuracy = accuracy_score(y_test, test_y_pred)
        
        mlflow.log_metric("train_accuracy", train_accuracy)
        mlflow.log_metric("test_accuracy", test_accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, test_y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [8]:
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42,class_weight=class_weights_dict)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for SVM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42,class_weight=class_weights_dict)

    # Log the best model with MLflow, passing the algo_name as "SVM"
    log_mlflow("SVM", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for SVM
run_optuna_experiment()

[I 2024-10-19 13:29:21,089] A new study created in memory with name: no-name-3a504c32-f7ba-40de-b879-c5358c86564e
[I 2024-10-19 13:33:58,292] Trial 0 finished with value: 0.5527041942604857 and parameters: {'C': 0.23453688461878391, 'kernel': 'poly'}. Best is trial 0 with value: 0.5527041942604857.
[I 2024-10-19 13:38:00,745] Trial 1 finished with value: 0.432257174392936 and parameters: {'C': 0.000636829263462398, 'kernel': 'rbf'}. Best is trial 0 with value: 0.5527041942604857.
[I 2024-10-19 13:48:58,576] Trial 2 finished with value: 0.6436258278145696 and parameters: {'C': 3.3532417215172865, 'kernel': 'poly'}. Best is trial 2 with value: 0.6436258278145696.
[I 2024-10-19 13:57:51,971] Trial 3 finished with value: 0.8349889624724062 and parameters: {'C': 3.9020553231280917, 'kernel': 'rbf'}. Best is trial 3 with value: 0.8349889624724062.
[I 2024-10-19 14:03:44,287] Trial 4 finished with value: 0.45626379690949226 and parameters: {'C': 0.07845896631052335, 'kernel': 'poly'}. Best is